# Project 4 Biodegradebility

The Goal of this project is to use QSAR-Data (Quantitative Structure Ability Relationship) from cemical Compounds and classify biodegradable and non biodegradable substances. Since compounds can last hundreds of years before being decomposed, degradability experiments will take time accordingly. This is where the approach of QSAR begins to shine. Just by looking at relatively quick to obtain molecular properties, the molecules behaviour (in this case biodegradability) can be estimated. Thus helping to ensure correct disposal of chemicals and saving the environment, while also reducing expensive longterm experiments.

In this Project i will:
* use my domain knowledge in chemistry to engineer sophisticated features
* explore multiple classification models
* apply and tune a logistic Regression Model

Link to Dataset Description : https://archive.ics.uci.edu/ml/datasets/QSAR+biodegradation#

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter('ignore') #ignore warning to imporve readability
pd.set_option("display.max_columns", 300) #make every column visible
%matplotlib inline

In [2]:
#custom functions
from functions import evaluate_classifier #plot confusion matrix and evaluate model using multible test metrics
from functions import plot_coefs #plot coefficients for Regression models

plt.rcParams["figure.figsize"] = [10,5] #setting for correct size of confusion matrix

#Initialize storing for results, in order to easily compare multiple models
results_dict_list = []
results_model_names_list = []

Load and Check Data

In [3]:
!ls #check that all needed files are in the current folder

Data                              README.md
LICENSE                           __pycache__
Pictures                          biodeg.csv
Presentation Biodegradability.pdf functions.py
Project 4 Biodegradables.ipynb    lgR_model_save.pickle


In [4]:
df = pd.read_csv("biodeg.csv", sep = ";", header = None)

In [5]:
df.head() #check columns

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,3.919,2.6909,0,0,0,0,0,31.4,2,0,0,0.000,3.106,2.550,9.002,0,0.960,1.142,0,0,0,1.201,0,0,0,0,1.932,0.011,0,0.000,4.489,0,0,0,0,2.949,1.591,0,7.253,0,0,RB
1,4.170,2.1144,0,0,0,0,0,30.8,1,1,0,0.000,2.461,1.393,8.723,1,0.989,1.144,0,0,0,1.104,1,0,0,0,2.214,-0.204,0,0.000,1.542,0,0,0,0,3.315,1.967,0,7.257,0,0,RB
2,3.932,3.2512,0,0,0,0,0,26.7,2,4,0,0.000,3.279,2.585,9.110,0,1.009,1.152,0,0,0,1.092,0,0,0,0,1.942,-0.008,0,0.000,4.891,0,0,0,1,3.076,2.417,0,7.601,0,0,RB
3,3.000,2.7098,0,0,0,0,0,20.0,0,2,0,0.000,2.100,0.918,6.594,0,1.108,1.167,0,0,0,1.024,0,0,0,0,1.414,1.073,0,8.361,1.333,0,0,0,1,3.046,5.000,0,6.690,0,0,RB
4,4.236,3.3944,0,0,0,0,0,29.4,2,4,0,-0.271,3.449,2.753,9.528,2,1.004,1.147,0,0,0,1.137,0,0,0,0,1.985,-0.002,0,10.348,5.588,0,0,0,0,3.351,2.405,0,8.003,0,0,RB


The DataFrame is missing column description. These have to be added from the Scource Website 

Since the Columns in the dataset are highly abrrivieated , a longer desciption is needed to truly understand the data. The description can originally be obtained here:

https://archive.ics.uci.edu/ml/datasets/QSAR+biodegradation#

For easy sorting functionality and a quick lookup the description is copied from the website and turned into a Pandas DataFrame using the following script:

In [6]:
f = open("Data/description_raw.txt", "r")
description_df = pd.DataFrame(columns = ["short", "description"])
info = "start"
while len(info):
    info = f.readline()
    if not info: 
        break
    info = info.split(") ",1)[1][:-2]
    short = info.split(": ",1)[0]
    #description = info.split(": ",1)[1]
    description_df.loc[len(description_df)] = info.split(": ",1)

The description is saved, to easily share it

In [11]:
#description_df.to_csv("data/description.csv", index=False)
description_df = pd.read_csv("Data/description.csv")

In [8]:
description_df.sort_values("short") #lookup table for Column descriptions

,short,description
23,B01[C-Br],Presence/absence of C - Br at topological dist...
24,B03[C-Cl],Presence/absence of C - Cl at topological dist...
28,B04[C-Br],Presence/absence of C - Br at topological dist...
7,C%,Percentage of C atoms
32,C-026,R--CX--R
3,F01[N-N],Frequency of N-N at topological distance 1
33,F02[C-N],Frequency of C - N at topological distance 2
10,F03[C-N],Frequency of C-N at topological distance 3
15,F03[C-O],Frequency of C - O at topological distance 3
4,F04[C-N],Frequency of C-N at topological distance 4


In [9]:
df.columns = description_df.short #assign describtions to column header

In [10]:
df.head()

short,SpMax_L,J_Dz(e),nHM,F01[N-N],F04[C-N],NssssC,nCb-,C%,nCp,nO,F03[C-N],SdssC,HyWi_B(m),LOC,SM6_L,F03[C-O],Me,Mi,nN-N,nArNO2,nCRX3,SpPosA_B(p),nCIR,B01[C-Br],B03[C-Cl],N-073,SpMax_A,Psi_i_1d,B04[C-Br],SdO,TI2_L,nCrt,C-026,F02[C-N],nHDon,SpMax_B(m),Psi_i_A,nN,SM6_B(m),nArCOOR,nX,experimental class
0,3.919,2.6909,0,0,0,0,0,31.4,2,0,0,0.000,3.106,2.550,9.002,0,0.960,1.142,0,0,0,1.201,0,0,0,0,1.932,0.011,0,0.000,4.489,0,0,0,0,2.949,1.591,0,7.253,0,0,RB
1,4.170,2.1144,0,0,0,0,0,30.8,1,1,0,0.000,2.461,1.393,8.723,1,0.989,1.144,0,0,0,1.104,1,0,0,0,2.214,-0.204,0,0.000,1.542,0,0,0,0,3.315,1.967,0,7.257,0,0,RB
2,3.932,3.2512,0,0,0,0,0,26.7,2,4,0,0.000,3.279,2.585,9.110,0,1.009,1.152,0,0,0,1.092,0,0,0,0,1.942,-0.008,0,0.000,4.891,0,0,0,1,3.076,2.417,0,7.601,0,0,RB
3,3.000,2.7098,0,0,0,0,0,20.0,0,2,0,0.000,2.100,0.918,6.594,0,1.108,1.167,0,0,0,1.024,0,0,0,0,1.414,1.073,0,8.361,1.333,0,0,0,1,3.046,5.000,0,6.690,0,0,RB
4,4.236,3.3944,0,0,0,0,0,29.4,2,4,0,-0.271,3.449,2.753,9.528,2,1.004,1.147,0,0,0,1.137,0,0,0,0,1.985,-0.002,0,10.348,5.588,0,0,0,0,3.351,2.405,0,8.003,0,0,RB


# Data Cleaning

TODO: write what you're analyzing here, each output should be explained!

In [ ]:
df.dtypes.value_counts()

In [ ]:
df.isnull().sum().sum()

TODO: Write full sentences/bullet points. always

Target Variable:

In [ ]:
df.replace(["RB","NRB"],[1,0], inplace = True)

In [ ]:
df.rename(columns = {"experimental class": "degradable"}, inplace = True);

In [ ]:
#Save Cleaned Data
#df.to_csv("biodeg_cleaned.csv", index=False)
#df = pd.read_csv("biodeg_cleaned.csv")

TODO: Acronyms have to be explained before you use them. always.

# EDA

TODO: headline/explanation missing - what are you doing here (and why)

In [ ]:
df.describe()

Check for Class Inbalance

In [ ]:
df["degradable"].value_counts(normalize = True)

In [ ]:
# NRB = 0 --> Not Biodegradable
# RB = 1 --> Biodegradable

--> Class inbalance acceptable

Check for Multicorrelation

In [ ]:
abs(df.corr()["degradable"]).sort_values(ascending = False).head(11)

In [ ]:
df.corr().applymap(lambda x: x if abs(x)>.90 else "")

Remove Multicorrelated Features

In [ ]:
corr_drop = ["SM6_L","SpMax_A","SM6_B(m)"]
corr_keep = list(set(df.columns)-set(corr_drop))

Comparision between feature distribution in the two target classes

In [ ]:
# Split into Target True and False
bio_df = df.loc[df["degradable"] == 1]
no_bio_df = df.loc[df["degradable"] == 0]

In [ ]:
features = corr_keep
for col in list(filter(lambda x: x != "diagnosis", features)):
    sns.distplot(bio_df[col] ,label = "degradable", color = "g")
    sns.distplot(no_bio_df[col], label = "non degradable" )
    plt.legend()
    plt.show()

Check for categorical features

In [ ]:
#If feature has more then 10 unique values it is probably not categorical, but continuous.
features = corr_keep
for col in features:
    print(col)
    valc = df[col].value_counts(normalize = True)
    if len(valc)<10:
        print(valc)
    else:
        print("probably not categorical")

## Results of EDA

The following Features lead to a high speration of the target classes. 
+ SpPosA_B(p)
+ HyWi_B(m)
+ C%
+ SpMax_B(m)
+ SpMax_L

# Feature Engineering

In [ ]:
df.shape

## Number of Heavy Meatals
This feature will be recoded to inform about heavy Metal atoms:
+ It is most likely that in degradable organic compounds there is only one heavy metal present (nHM =1) in as a single central atom in a chemical complex bound.
+ If nHM exeeds one a compound will be either anorganic or of high toxicity and therefore lead to low biodegradability. (nHM >1)

In [ ]:
def encode_nHM(x):
    if x == 0:
        return "light"
    if x == 1:
        return "functional"
    if x >1:
        return "heavy"

In [ ]:
df["nHM_enc"] = df["nHM"].apply(encode_nHM)

Grouping shows that there is in fact a certain seperation of target classes in using this feature.

In [ ]:
df.groupby(["nHM_enc","degradable"]).count()

In [ ]:
nHM_dummies = pd.get_dummies(df["nHM_enc"],"nHM").drop("nHM_light",axis = 1)
df = pd.concat([df,nHM_dummies], axis = 1).drop("nHM_enc", axis = 1)

## Molecule Shape
The number of terminal C-Atoms informs about the overall shape of a moleculy, highly branched meolekules are expected to be less degradable, because the branches make it harder for enzymes to connect to the molecule and deconstruct it.

In [ ]:
def shape_class(x):
    if x == 0:
        return "ring"
    if x == 1:
        return "semi_ring"
    if x == 2:
        return "linear"
    if x >2:
        return "branched"

In [ ]:
df["molShape"] = df["nCp"].apply(shape_class)

Grouping shows that espeacially branched and ring shaped molecules provide a good seperation between target classes.

In [ ]:
df.groupby(["molShape","degradable"]).count()

In [ ]:
mS_dummies = pd.get_dummies(df["molShape"],"mS").drop("mS_semi_ring",axis = 1)
df = pd.concat([df,mS_dummies], axis = 1).drop("molShape",axis = 1)

## Esther
Having a Esther group in the molecule provides an easy breaking point due to hydrolysis. Therefore degradability should improve in esthers.

In [ ]:
df["esther"] = df.nArCOOR.apply(lambda x: 1 if x>0 else x)

Grouping shows s seperation but the data lecks observations with molecule containing esther groups. Therefor the effect of this festure will be hardly recognisable

In [ ]:
df.groupby(["esther","degradable"]).count()

In [ ]:
df.dtypes.value_counts()

Save/Load engineered DataFrame

In [ ]:
#df.to_csv("Data/df_engineered.csv", index=False)
#df = pd.read_csv("Data/df_engineered.csv")

In [ ]:
df.head()

## Train_Test_Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#remove colinear columns as identified in EDA
keep = list(set(df.columns) - set(corr_drop))

Define Target and Feature Variables and perform standard 80/20 split.

In [ ]:
y = df.degradable
X = df[keep].drop("degradable", axis = 1)
X_train, X_test, y_train , y_test = train_test_split(X,y ,random_state=42 , test_size = 0.3)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape , y_test.shape

## Transformation

In [ ]:
from sklearn.preprocessing import StandardScaler

### Poly

Generate Interaction Terms

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(2,include_bias = False)
poly.fit(X_train)
X_train_poly = (pd.DataFrame(poly.transform(X_train),columns = poly.get_feature_names(X_train.columns)))
X_test_poly = (pd.DataFrame(poly.transform(X_test),columns = poly.get_feature_names(X_train.columns)))

Scale Features to improve Model performance

In [ ]:
scaler_poly = StandardScaler()
scaler_poly.fit(X_train_poly)
X_train_poly = pd.DataFrame(scaler_poly.transform(X_train_poly),columns = X_train_poly.columns)
X_test_poly = pd.DataFrame(scaler_poly.transform(X_test_poly),columns = X_train_poly.columns)

### Linear

Scale Features to improve Model performance

In [ ]:
scaler_lin = StandardScaler()
scaler_lin.fit(X_train)
X_train = pd.DataFrame(scaler_lin.transform(X_train),columns = X_train.columns)
X_test = pd.DataFrame(scaler_lin.transform(X_test),columns = X_train.columns)

In [ ]:
lin_cols_all = X_train.columns

#X_train.to_csv("Data/X_train_lin_scaled.csv", index=False)
#X_test.to_csv("Data/X_test_lin_scaled.csv", index=False)

# Feature Selection

Feature Selection will be performed using Lasso Penalty in a Logistic Regression Classifier Model.
The target metric wills be:
+ Precission (because predicting biodegradability when the molecule is not (False Positive) is worse than predicting nondegradability allthough the molecule is degradable.
+ Specificity (to compare Model performance with Values in the Scource Paper)
+ Sensitivity (to compare Model performance with Values in the Scource Paper)
+ F1 (to get an overall scoring)

Features are dropped if dropping them does not significantly lower model performance.

In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

## Basemodel

### Logistic Regression

In [ ]:
logReg_base = LogisticRegression(random_state=42)

logReg_base.fit(X_train,y_train)
logReg_base_pred = logReg_base.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,logReg_base_pred, normalize=True))
results_model_names_list.append("LogR_base")

# Modeling

## Linear

## Logistic Regression

In [ ]:
logReg_base_l1 = LogisticRegression(solver = "saga", penalty = "l1", C = 0.7, random_state=42)

logReg_base_l1.fit(X_train,y_train)
logReg_base_l1_pred = logReg_base_l1.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,logReg_base_l1_pred, normalize=True))
drop_l1 = plot_coefs(X_train,logReg_base_l1)
results_model_names_list.append("LogR_lin_l1")

In [ ]:
l1_keep = list(set(X_train.columns) - set(drop_l1))
l1_keep

## Random Forrest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest_base = RandomForestClassifier(n_estimators=100, random_state=42)

forest_base.fit(X_train,y_train)
forest_base_pred = forest_base.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,forest_base_pred, normalize=True));
results_model_names_list.append("RandFor_lin")

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNN_base = KNeighborsClassifier(n_neighbors = 13)

KNN_base.fit(X_train,y_train)
KNN_base_pred = KNN_base.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,KNN_base_pred, normalize=True));
results_model_names_list.append("KNN_lin")

## Ensemble

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

In [ ]:
clf1 = logReg_base_l1
clf2 = forest_base
clf3 = KNN_base
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('KNN', clf3)], voting='hard')
eclf.fit(X_train,y_train)

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'KNN', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1')
    print("F1: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
eclf_base_pred = eclf.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,eclf_base_pred, normalize=True));
results_model_names_list.append("Ensemble_lin")

## Poly

## Logistic Regression

In [ ]:
X_train = X_train_poly.copy()
X_test = X_test_poly.copy()

In [ ]:
logReg_base_l1_poly = LogisticRegression(solver = "saga", penalty = "l1", C = 1, random_state=42)

logReg_base_l1_poly.fit(X_train_poly,y_train)
logReg_base_l1_poly_pred = logReg_base_l1_poly.predict(X_test_poly)
results_dict_list.append(evaluate_classifier(y_test,logReg_base_l1_poly_pred, normalize=True))
drop_l1_poly = plot_coefs(X_train_poly,logReg_base_l1_poly)
results_model_names_list.append("LogR_poly")

## Random Forrest

In [ ]:

forest_base = RandomForestClassifier(n_estimators=100, random_state=42)

forest_base.fit(X_train,y_train)
forest_base_pred = forest_base.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,forest_base_pred, normalize=True));
results_model_names_list.append("RandFor_poly")

In [ ]:
rfc_featimp_df = pd.DataFrame(forest_base.feature_importances_, index = X_train.columns)
sorted_rfc = rfc_featimp_df.sort_values(by = 0, ascending = False)
rfc_add = list(sorted_rfc[sorted_rfc[0]>0.01].head(3).index)
rfc_add

## KNN

In [ ]:

KNN_base = KNeighborsClassifier(n_neighbors = 13)

KNN_base.fit(X_train,y_train)
KNN_base_pred = KNN_base.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,KNN_base_pred, normalize=True));
results_model_names_list.append("KNN_poly")

## Ensemble

In [ ]:

clf1 = logReg_base_l1
clf2 = forest_base
clf3 = KNN_base
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('KNN', clf3)], voting='hard')
eclf.fit(X_train,y_train)

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'KNN', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1')
    print("F1: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
eclf_base_pred = eclf.predict(X_test)
results_dict_list.append(evaluate_classifier(y_test,eclf_base_pred, normalize=True));
results_model_names_list.append("Ensemble_poly")

# Model Selection

In [ ]:
from functions import compare_models

Compare model score metrics

In [ ]:
out_df = compare_models(results_dict_list, results_model_names_list )
out_df["sum"] = out_df.iloc[:,0:4].sum(axis = 1)
out_df.drop("confusion_matrix", axis = 1).applymap(lambda x: round(x,2)).sort_values(by = "sum", ascending = False)

Plot results

In [ ]:
out_df = out_df.sort_values(by = "sum", ascending = False)

In [ ]:
res_cols = ["accuracy","precission","sensitivity","f_1"]

for col in res_cols:
    plt.plot(out_df.index, out_df[col], label = col)
    plt.xticks(rotation=90)
    plt.legend()

## Result of Model Selection
Both Logistic Regression models without polynomial Features perform quite well, thus Logistic Regression is chosen for optimization

# Model Optimization

Select linear Features, that were not restricted to a coefficient of zero by lasso selection and reselect Training and Test Feature accordingly.

In [ ]:
features = list(set(l1_keep).union((lin_cols_all)))
#features

In [ ]:
X_train = X_train_poly[features]
X_test = X_test_poly[features]

## Gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

A custom scoring is chosen to best fit all of the specified metrics

In [ ]:
def custom_scorer(y_true,y_pred):
    f1 = metrics.f1_score(y_true,y_pred)
    acc = metrics.accuracy_score(y_true,y_pred)
    prec = metrics.precision_score(y_true,y_pred)
    sens = metrics.recall_score(y_true,y_pred)
    return f1+acc+prec+sens

Initialize and run gridsearch

In [ ]:
lgR_20 = LogisticRegression(random_state = 42)
param_grid = {"solver":["liblinear","saga"],
          "penalty":["l1","l2"],
          "C":np.arange(0.1,1,0.05),
             }

In [ ]:
grid_logR = GridSearchCV(estimator = lgR_20,
                         param_grid = param_grid,
                         scoring = make_scorer(custom_scorer),
                         cv = 5,
                         verbose = 4,
                         n_jobs = -1)

In [ ]:
grid_logR.fit(X_train,y_train)

In [ ]:
grid_logR.best_score_

In [ ]:
grid_logR_pred = grid_logR.best_estimator_.predict(X_test)
evaluate_classifier(grid_logR_pred,y_test, normalize=True);

In [ ]:
grid_logR.best_params_

# Final Model

Refit Model and Evaluate

In [ ]:
lgR_2 = LogisticRegression(solver = "liblinear", penalty = "l2", C = 0.9, random_state=42)
lgR_2.fit(X_train,y_train)
lgR_2_pred = lgR_2.predict(X_test)
results_dict_list.append(evaluate_classifier(lgR_2_pred,y_test, normalize=True))

Allthough Model performance did not improve with running the gridseach, the cross validation used provied a more robust model. Hence the Final model as seen above is used.

## Inspect final feature importance

In [ ]:
zeros = plot_coefs(X_train,lgR_2)

In [ ]:
zeros

In [ ]:
zeros_df = plot_coefs(X_train,lgR_2,return_nulls = False);

Rank top 20 most important features and display them for visualization

In [ ]:
top_20_df = zeros_df.sort_values(by = "abs", ascending = False).head(20)

In [ ]:
top_20_df["rank"] = list(range(1,21))

In [ ]:
top_20_df[["rank","coeff"]]

Save Model

In [ ]:
import pickle

In [ ]:
#with open("lgR_model_save.pickle" ,"wb") as f:
    #pickle.dump(lgR_2,f) 